In [15]:
import os
import pandas as pd
import re
import numpy as np
import collections
from operator import itemgetter
import pprint as pp
import re
import nltk
from nltk.corpus import stopwords
import nltk.data
import time


#data from http://times.cs.uiuc.edu/~wang296/Data/
files = os.listdir('./Review_Texts')
#print(os.listdir('./Review_Texts'))
print(len(files))

1759


In [16]:
cats = ['Rooms', 'Date', 'Location', 'Service', 'Business service', 'Author', 'Check in / front desk', 'No. Helpful', 'Cleanliness', 'Content', 'Value', 'No. Reader', 'Overall']
data = pd.DataFrame(columns=cats)

In [17]:
def addFileToData(filename, data):
    intColumns = ['No. Reader', 'No. Helpful', 'Cleanliness','Check in / front desk', 'Value', 'Overall', 'Service', 'Business service', 'Rooms', 'Location']
    characterThreshold = 60
    with open(filename, 'r') as content_file:
        content = content_file.read()
        
    reviews = content.split("\n\n")
    for r in reviews:
        thisReview = pd.Series([None]*len(cats), cats)
        splt = r.split("\n")
        for s in splt:
            for c in cats:
                if "<"+c+">" in s:
                    value = s.replace('<'+c+'>', '')
                    if c in intColumns:
                        value = int(value)
                    if value == -1: #we dont want -1 as this is going to mess up averaging, take np.nan
                        value = np.nan

                    if c == "Content":
                        value = value.lower()

                    thisReview[c] = value
                    
        if not thisReview["Content"] == None and len(thisReview["Content"]) > characterThreshold:
            #only add if theres content and its long enough
            data = data.append(thisReview, ignore_index=True)
    return data
        
data = addFileToData('./Review_Texts/hotel_100504.dat', data) 
data = addFileToData('./Review_Texts/hotel_72572.dat', data) 


In [18]:
print(data.shape)
data.head()

(199, 13)


,Rooms,Date,Location,Service,Business service,Author,Check in / front desk,No. Helpful,Cleanliness,Content,Value,No. Reader,Overall
0,5.0,"Dec 23, 2008",5.0,5.0,NaN,selizabethm,5.0,NaN,5.0,wonderful time- even with the snow! what a gre...,4.0,NaN,5.0
1,4.0,"Nov 13, 2008",5.0,5.0,NaN,IndieLady,5.0,NaN,4.0,"lovely hotel, unique decor, friendly front des...",5.0,NaN,4.0
2,4.0,"Nov 11, 2008",3.0,NaN,4,Hilobb,5.0,NaN,4.0,"nice hotel, expensive parking we got a good de...",4.0,NaN,4.0
3,5.0,"Nov 4, 2008",5.0,5.0,5,Chianti_girl24,5.0,NaN,5.0,fabulous hotel location and service are great....,5.0,NaN,5.0
4,NaN,"Oct 18, 2008",NaN,NaN,NaN,hothearted,NaN,2,NaN,"loved the monaco! staff was amazing, with a sm...",NaN,2,5.0


In [19]:
#creating the vocab of all the words

def lintWord(w):
    regex = re.compile(r'(,|\(|\)|!|:|$|\.)')
    w = re.sub(regex, '', w)
    return w


allWords = ""
for r in data["Content"]:
    #add word to big content string
    allWords += r + " "
    
#split the string at spaces, keep only unique
words = set(allWords.split(" "))


vocab = list(set([lintWord(w) for w in words if not w in stopwords.words("english")]))

#n eed to remove stopwords again because some of them may have had punctuation 
# at the end and didnt get caught the first time
vocab = [w for w in vocab if not w in stopwords.words("english") and len(w) > 2]

#vocab

C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:18: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
C:\Users\Vinayak11\Anaconda\lib\site-packages\ipykernel\__main__.py:22: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [ ]:
#tokenizer to split sentences
from nltk.tokenize import RegexpTokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
tokenizer2 = RegexpTokenizer(r'\w+')

#Stemmer defined for stemming words
from nltk.stem.porter import *
stemmer = PorterStemmer()

# -----------------------------------------------------------------------------------------
#                                    LDA IMPLEMENTATION
# -----------------------------------------------------------------------------------------

In [ ]:
# initial seeds from LARA paper
from gensim import corpora, models
seeds = {"Value" : ["value", "price", "quality","worth"],
         "Rooms" : ["room", "suite", "view", "bed"],
         "Location" : ["location", "traffic", "minute", "restaurant"],
         "Cleanliness" : ["clean", "dirty", "maintain", "smell"],
         "Check in / front desk": ["stuff", "check", "help", "reservation"],
         "Service" : ["service", "food", "breakfast", "buffet"],
         "Business service" : ["business", "center", "computer", "internet"]
        }


def aspectSegmentation(reviews, aspects, vocab=[], threshold=0, iterationLimit=3):
    #when we have the top chi-squared rated keywords, how many do we take
    keywordsToTake = 3
    
    #bootstrap iterations
    for i in range(0, iterationLimit):
        
        #print our current aspects
        print("begin bootstrapping iteration, aspect keywords: ")
        pp.pprint(aspects)
        print("\n\n\n")
        labeledSentences = []
        for r in reviews:
            
            #use the pickle tokenizer to split sentences
                 
            sentences = tokenizer.tokenize(r.decode('utf-8'))

                
            for s in sentences:
                
                theseAspects = collections.defaultdict(int)
                maxAspect = (0, "None")
                
                #for each aspect count how many times one of those aspect words appears
                for a in aspects:
                    for word in aspects[a]:
                        if " "+word+" " in s:
                            theseAspects[a] += 1
                
                #find the max occuring aspect for each sentence, take multiple if ties
                for a in theseAspects:
                    if theseAspects[a] > maxAspect[0]:
                        maxAspect = (theseAspects[a], a)
                    if theseAspects[a] == maxAspect[0] and a not in maxAspect:
                        #label it with multiple aspects
                        maxAspect = maxAspect + (a, )
                
                #add the sentence with labels
                labeledSentences.append((s, maxAspect[1:]))
            
        LDAForAspects = collections.defaultdict(list)
        
# obtain dictionary of tokenized sentences corresponding to each aspect
        for a in aspects:
            for s in labeledSentences:
                sentenceText=s[0]
                sentenceAspects=s[1]
                
                if a in sentenceAspects:
                    sentenceTextTokens=tokenizer2.tokenize(sentenceText)
                    sentenceTextTokensNS=[i for i in sentenceTextTokens if not i in stopwords.words("english") and i>2]
                    sentenceTextTokensNS_stemmed = [stemmer.stem(i) for i in sentenceTextTokensNS]
                    LDAForAspects[a].extend(sentenceTextTokensNS)

#Implement LDA to obtain topics
        SentenceWords=[]
        dictionary={}
        corpus={}
        for a in aspects:
            SentenceWords.append(LDAForAspects[a])
            dictionary[a]=corpora.Dictionary([LDAForAspects[a]])
            corpus[a] = [dictionary[a].doc2bow([text]) for text in LDAForAspects[a]]
            ldamodel = models.ldamodel.LdaModel(corpus[a], num_topics=1, id2word = dictionary[a], passes=20)
            print a+":"
            print(ldamodel.print_topics(num_topics=1, num_words=5))
            X=ldamodel.get_topic_terms(0,5)
            i=0
            while i<len(X):
                temp=list(dictionary[a].token2id.keys())[list(dictionary[a].token2id.values()).index(X[i][0])]
#     Checking for duplicates
                if temp not in aspects[a]:  
                    aspects[a].append(temp)
                i=i+1
            print aspects[a]
    return labeledSentences


start = time.time()
sentencesWLabels = aspectSegmentation(data["Content"], seeds, vocab)
end = time.time()

print("done, time taken:", end-start)

begin bootstrapping iteration, aspect keywords: 
{'Business service': ['business', 'center', 'computer', 'internet'],
 'Check in / front desk': ['stuff', 'check', 'help', 'reservation'],
 'Cleanliness': ['clean', 'dirty', 'maintain', 'smell'],
 'Location': ['location', 'traffic', 'minute', 'restaurant'],
 'Rooms': ['room', 'suite', 'view', 'bed'],
 'Service': ['service', 'food', 'breakfast', 'buffet'],
 'Value': ['value', 'price', 'quality', 'worth']}




Service:
[(0, u'0.041*service + 0.021*hotel + 0.019*room + 0.018*breakfast + 0.012*great')]
['service', 'food', 'breakfast', 'buffet', u'hotel', u'room']
Business service:
[(0, u'0.021*business + 0.020*hotel + 0.014*internet + 0.008*room + 0.008*wireless')]
['business', 'center', 'computer', 'internet', u'hotel']
Cleanliness:
[(0, u'0.046*clean + 0.023*room + 0.021*hotel + 0.014*rooms + 0.012*staff')]
['clean', 'dirty', 'maintain', 'smell', u'room', u'hotel']
Check in / front desk:
[(0, u'0.019*check + 0.012*help + 0.010*desk + 0.010*

In [ ]:
numToShow = 20
count = 0
for s in sentencesWLabels:
    if 'None' not in s[1]:
        print(s)
        print("\n")
        count += 1
    if count > numToShow:
        break